# 模拟浏览器爬取数据

## 1. 导入测试模块框架selenium


### 1.1 下载selenium测试框架和依赖浏览器驱动包


In [ ]:
!pip install selenium

In [ ]:
!mkdir ~/Chrome
!wget -P ~/Chrome https://download-installer.cdn.mozilla.net/pub/firefox/releases/73.0.1/linux-x86_64/zh-CN/firefox-73.0.1.tar.bz2 
# !wget -P ~/Chrome/ http://chromedriver.storage.googleapis.com/81.0.4044.20/chromedriver_linux64.zip

### 1.2 解压谷歌浏览器驱动

In [ ]:
!tar -xjvf ~/Chrome/firefox-73.0.1.tar.bz2 -C ~/Chrome
# !unzip ~/Chrome/chromedriver_linux64.zip -d ~/Chrome

In [ ]:
!wget -P ~/Chrome https://github.com/mozilla/geckodriver/releases/download/v0.25.0/geckodriver-v0.25.0-linux64.tar.gz
!tar -zxvf ~/Chrome/geckodriver-v0.25.0-linux64.tar.gz -C ~/Chrome/firefox/

### 1.3 导入测试框架

In [ ]:
import selenium
from selenium import webdriver #测试框架模拟浏览器

### 1.4 嵌入正则表达式

In [ ]:
import re
def regex_webside(text):
    restr = r'<span class="total"> (\d+) </span>' # 正则表达式匹配
    regex =  re.compile(restr,re.I) # 预编译，re.I 不分大小写
    result = regex.findall(pagesource)
    return result

### 1.5 调用selenium框架爬取网页信息

In [ ]:

# 创建chrome参数对象
# opt = selenium.webdriver.ChromeOptions()
from selenium.webdriver.firefox.options import Options
options = Options()
options.add_argument('--headless')
def get_webside(keywords):
    url = "https://xiaoyuan.zhaopin.com/search/jn=4&kw="+ keywords
    driver = selenium.webdriver.Firefox(options=options, # 传递无页面响应参数
                                        firefox_binary=r'/home/nbuser/Chrome/firefox/firefox', # 传递火狐浏览器位置
                                       executable_path = r'/home/nbuser/Chrome/firefox/geckodriver') # 调用火狐浏览器依赖驱动
    driver.get(url)
    pagesource = driver.page_source# 获取爬取页面信息
    num = regex_webside(pagesource)
    return num

In [ ]:
get_webside("python")

# 2. 封装爬虫框架，爬取不同页面信息

## 2.1 封装正则表达式

In [ ]:
import re
def regex_webside_find(restr,text,** args):
    ''' 通过正则表达式查找网页信息
        - param restr: 自定义要匹配的正则表达式
        - param text: 要进行匹配的文本
        - 返回查找出的全部子串
    '''
    regex =  re.compile(restr,re.I) # 预编译，re.I 不分大小写
    result = regex.findall(pagesource)
    if len(result) == 0:
        return "NULL"
    return result

## 2.2 封装爬虫框架

In [ ]:
import selenium
from selenium import webdriver #测试框架模拟浏览器# 创建chrome参数对象
from selenium.webdriver.firefox.options import Options

def get_webside(url,data,**arg):
    ''' 使用selenium模拟浏览器爬虫
        - param url: 需要爬取的网页url
        - param data: 可能需要传递的数据，以键值对的形式传入
    '''
    # 设置无页面响应浏览器     
    options = Options()
    options.add_argument('--headless')
    # 遍历键值对，并将其加入进url尾部
    for key in data:
        url += "&" + key + "=" + data[key]
        
#     url = "https://xiaoyuan.zhaopin.com/search/jn=4&kw="+ keywords
    driver = selenium.webdriver.Firefox(options=options, # 传递无页面响应参数
                                        firefox_binary=r'/home/nbuser/Chrome/firefox/firefox', # 传递火狐浏览器位置
                                       executable_path = r'/home/nbuser/Chrome/firefox/geckodriver') # 调用火狐浏览器依赖驱动
    driver.get(url)
#     pagesource = driver.page_source# 获取爬取页面信息
    return driver

## 2.3 爬取其他网页信息（马蜂窝获取页面景点名称）

In [ ]:
def get_city_page_num(city):
    ''' 通过传入城市名称获取当前城市显示的景点页数
        - param city: 城市名称
        - return 返回当前城市的景点页数
    '''
    regx = r'data-page="(\d+)" rel="nofollow">末页</a>' #设置用于获取页数的正则表达式
    url = "http://www.mafengwo.cn/search/q.php?" #马蜂窝访问旅游景点url入口
    data = {
        "t":"pois",
        "p":"1",
        "q":city # 设置要查询的城市参数
    }
    response = get_webside(url = url,data = data)
    num = regex_webside_find(restr=regx,text = response.page_source)
    if num == "NULL":
        return "信息获取失败"
    else:
        return num[0]

In [ ]:
city_list = ['衡阳','长沙','湘潭']
for city in city_list:
    print city+"的页数:"+get_city_page_num(city)
